# Image classification with modern MLP models

### This example implements three modern attention-free, multi-layer perceptron (MLP) based models for image classification, demonstrated on the CIFAR-100 dataset:
    The MLP-Mixer model, by Ilya Tolstikhin et al., based on two types of MLPs.
    The FNet model, by James Lee-Thorp et al., based on unparameterized Fourier Transform.
    The gMLP model, by Hanxiao Liu et al., based on MLP with gating.
The purpose of the example is not to compare between these models, as they might perform differently on different datasets with well-tuned hyperparameters. Rather, it is to show simple implementations of their main building blocks. 

In [1]:
pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


## Setup 

In [3]:
!pip install tensorflow 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import all libraries that we need 

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

## Prepare the data

In [5]:
num_classes = 100
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

169001437/169001437 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


## Configure the hyperparameters

In [6]:
weight_decay = 0.0001
batch_size = 128
num_epochs = 50
dropout_rate = 0.2
image_size = 64  # We'll resize input images to this size.
patch_size = 8  # Size of the patches to be extracted from the input images.
num_patches = (image_size // patch_size) ** 2  # Size of the data array.
embedding_dim = 256  # Number of hidden units.
num_blocks = 4  # Number of blocks.

print(f"Image size: {image_size} X {image_size} = {image_size ** 2}")
print(f"Patch size: {patch_size} X {patch_size} = {patch_size ** 2} ")
print(f"Patches per image: {num_patches}")
print(f"Elements per patch (3 channels): {(patch_size ** 2) * 3}")

Image size: 64 X 64 = 4096
Patch size: 8 X 8 = 64 
Patches per image: 64
Elements per patch (3 channels): 192


## Build a classification model

### implement a method that builds a classifier given the processing blocks.

 

In [7]:
def build_classifier(blocks, positional_encoding=False):
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size, num_patches)(augmented)
    # Encode patches to generate a [batch_size, num_patches, embedding_dim] tensor.
    x = layers.Dense(units=embedding_dim)(patches)
    if positional_encoding:
        positions = tf.range(start=0, limit=num_patches, delta=1)
        position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=embedding_dim
        )(positions)
        x = x + position_embedding
    # Process x using the module blocks.
    x = blocks(x)
    # Apply global average pooling to generate a [batch_size, embedding_dim] representation tensor.
    representation = layers.GlobalAveragePooling1D()(x)
    # Apply dropout.
    representation = layers.Dropout(rate=dropout_rate)(representation)
    # Compute logits outputs.
    logits = layers.Dense(num_classes)(representation)
    # Create the Keras model.
    return keras.Model(inputs=inputs, outputs=logits)

## Define an experiment

### implement a utility function to compile, train, and evaluate a given model. 

In [9]:
def run_experiment(model):
    # Create Adam optimizer with weight decay.
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay,
    )
    # Compile the model.
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="acc"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top5-acc"),
        ],
    )
    # Create a learning rate scheduler callback.
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5
    )
    # Create an early stopping callback.
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[early_stopping, reduce_lr],
    )

    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    # Return history to plot learning curves.
    return history

## Use data augementation

In [10]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)

## Implement patch extraction as a layer

In [11]:
class Patches(layers.Layer):
    def __init__(self, patch_size, num_patches):
        super(Patches, self).__init__()
        self.patch_size = patch_size
        self.num_patches = num_patches

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        return patches

## The MLP-Mixer model

### Implement the MLP-Mixer module 

#### The MLP-Mixer is an architecture based exclusively on multi-layer perceptrons (MLPs), that contains two types of MLP layers:

  One applied independently to image patches, which mixes the per-location features.
  
  The other applied across patches (along channels), which mixes spatial information. 

In [12]:
class MLPMixerLayer(layers.Layer):
    def __init__(self, num_patches, hidden_units, dropout_rate, *args, **kwargs):
        super(MLPMixerLayer, self).__init__(*args, **kwargs)

        self.mlp1 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=num_patches),
                layers.Dropout(rate=dropout_rate),
            ]
        )
        self.mlp2 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=embedding_dim),
                layers.Dropout(rate=dropout_rate),
            ]
        )
        self.normalize = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        # Apply layer normalization.
        x = self.normalize(inputs)
        # Transpose inputs from [num_batches, num_patches, hidden_units] to [num_batches, hidden_units, num_patches].
        x_channels = tf.linalg.matrix_transpose(x)
        # Apply mlp1 on each channel independently.
        mlp1_outputs = self.mlp1(x_channels)
        # Transpose mlp1_outputs from [num_batches, hidden_dim, num_patches] to [num_batches, num_patches, hidden_units].
        mlp1_outputs = tf.linalg.matrix_transpose(mlp1_outputs)
        # Add skip connection.
        x = mlp1_outputs + inputs
        # Apply layer normalization.
        x_patches = self.normalize(x)
        # Apply mlp2 on each patch independtenly.
        mlp2_outputs = self.mlp2(x_patches)
        # Add skip connection.
        x = x + mlp2_outputs
        return x

### Build, train, and evaluate the MLP-Mixer model 

In [13]:
mlpmixer_blocks = keras.Sequential(
    [MLPMixerLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.005
mlpmixer_classifier = build_classifier(mlpmixer_blocks)
history = run_experiment(mlpmixer_classifier)

Epoch 1/50
352/352 [==============================] - 50s 113ms/step - loss: 3.8753 - acc: 0.1106 - top5-acc: 0.3162 - val_loss: 3.5713 - val_acc: 0.1764 - val_top5-acc: 0.4222 - lr: 0.0050
Epoch 2/50
352/352 [==============================] - 39s 111ms/step - loss: 3.3633 - acc: 0.1896 - top5-acc: 0.4644 - val_loss: 3.2103 - val_acc: 0.2394 - val_top5-acc: 0.5170 - lr: 0.0050
Epoch 3/50
352/352 [==============================] - 39s 110ms/step - loss: 3.1404 - acc: 0.2321 - top5-acc: 0.5258 - val_loss: 3.0634 - val_acc: 0.2638 - val_top5-acc: 0.5548 - lr: 0.0050
Epoch 4/50
352/352 [==============================] - 38s 108ms/step - loss: 2.9857 - acc: 0.2628 - top5-acc: 0.5645 - val_loss: 2.9254 - val_acc: 0.2772 - val_top5-acc: 0.5884 - lr: 0.0050
Epoch 5/50
352/352 [==============================] - 38s 107ms/step - loss: 2.8911 - acc: 0.2826 - top5-acc: 0.5848 - val_loss: 2.8329 - val_acc: 0.2932 - val_top5-acc: 0.6182 - lr: 0.0050
Epoch 6/50
352/352 [==============================

## The FNet model

### Implement the FNet module 

In [14]:
class FNetLayer(layers.Layer):
    def __init__(self, num_patches, embedding_dim, dropout_rate, *args, **kwargs):
        super(FNetLayer, self).__init__(*args, **kwargs)

        self.ffn = keras.Sequential(
            [
                layers.Dense(units=embedding_dim),
                tfa.layers.GELU(),
                layers.Dropout(rate=dropout_rate),
                layers.Dense(units=embedding_dim),
            ]
        )

        self.normalize1 = layers.LayerNormalization(epsilon=1e-6)
        self.normalize2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        # Apply fourier transformations.
        x = tf.cast(
            tf.signal.fft2d(tf.cast(inputs, dtype=tf.dtypes.complex64)),
            dtype=tf.dtypes.float32,
        )
        # Add skip connection.
        x = x + inputs
        # Apply layer normalization.
        x = self.normalize1(x)
        # Apply Feedfowrad network.
        x_ffn = self.ffn(x)
        # Add skip connection.
        x = x + x_ffn
        # Apply layer normalization.
        return self.normalize2(x)

#### The FNet uses a similar block to the Transformer block. However, FNet replaces the self-attention layer in the Transformer block with a parameter-free 2D Fourier transformation layer:

  One 1D Fourier Transform is applied along the patches.

  One 1D Fourier Transform is applied along the channels.

### Build, train, and evaluate the FNet model 

In [15]:
fnet_blocks = keras.Sequential(
    [FNetLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.001
fnet_classifier = build_classifier(fnet_blocks, positional_encoding=True)
history = run_experiment(fnet_classifier)

Epoch 1/50
352/352 [==============================] - 43s 109ms/step - loss: 4.1378 - acc: 0.0729 - top5-acc: 0.2300 - val_loss: 3.8528 - val_acc: 0.1074 - val_top5-acc: 0.3228 - lr: 0.0010
Epoch 2/50
352/352 [==============================] - 38s 108ms/step - loss: 3.7183 - acc: 0.1286 - top5-acc: 0.3526 - val_loss: 3.5733 - val_acc: 0.1564 - val_top5-acc: 0.3988 - lr: 0.0010
Epoch 3/50
352/352 [==============================] - 38s 107ms/step - loss: 3.4625 - acc: 0.1714 - top5-acc: 0.4285 - val_loss: 3.2876 - val_acc: 0.2050 - val_top5-acc: 0.4762 - lr: 0.0010
Epoch 4/50
352/352 [==============================] - 39s 111ms/step - loss: 3.2690 - acc: 0.2056 - top5-acc: 0.4838 - val_loss: 3.1228 - val_acc: 0.2364 - val_top5-acc: 0.5124 - lr: 0.0010
Epoch 5/50
352/352 [==============================] - 38s 108ms/step - loss: 3.1324 - acc: 0.2296 - top5-acc: 0.5207 - val_loss: 3.0093 - val_acc: 0.2500 - val_top5-acc: 0.5512 - lr: 0.0010
Epoch 6/50
352/352 [==============================

## The gMlp model

### Implement the gMLP module

#### The gMLP is a MLP architecture that features a Spatial Gating Unit (SGU). The SGU enables cross-patch interactions across the spatial (channel) dimension, by:

Transforming the input spatially by applying linear projection across patches (along channels).
Applying element-wise multiplication of the input and its spatial transformation.

In [16]:
class gMLPLayer(layers.Layer):
    def __init__(self, num_patches, embedding_dim, dropout_rate, *args, **kwargs):
        super(gMLPLayer, self).__init__(*args, **kwargs)

        self.channel_projection1 = keras.Sequential(
            [
                layers.Dense(units=embedding_dim * 2),
                tfa.layers.GELU(),
                layers.Dropout(rate=dropout_rate),
            ]
        )

        self.channel_projection2 = layers.Dense(units=embedding_dim)

        self.spatial_projection = layers.Dense(
            units=num_patches, bias_initializer="Ones"
        )

        self.normalize1 = layers.LayerNormalization(epsilon=1e-6)
        self.normalize2 = layers.LayerNormalization(epsilon=1e-6)

    def spatial_gating_unit(self, x):
        # Split x along the channel dimensions.
        # Tensors u and v will in th shape of [batch_size, num_patchs, embedding_dim].
        u, v = tf.split(x, num_or_size_splits=2, axis=2)
        # Apply layer normalization.
        v = self.normalize2(v)
        # Apply spatial projection.
        v_channels = tf.linalg.matrix_transpose(v)
        v_projected = self.spatial_projection(v_channels)
        v_projected = tf.linalg.matrix_transpose(v_projected)
        # Apply element-wise multiplication.
        return u * v_projected

    def call(self, inputs):
        # Apply layer normalization.
        x = self.normalize1(inputs)
        # Apply the first channel projection. x_projected shape: [batch_size, num_patches, embedding_dim * 2].
        x_projected = self.channel_projection1(x)
        # Apply the spatial gating unit. x_spatial shape: [batch_size, num_patches, embedding_dim].
        x_spatial = self.spatial_gating_unit(x_projected)
        # Apply the second channel projection. x_projected shape: [batch_size, num_patches, embedding_dim].
        x_projected = self.channel_projection2(x_spatial)
        # Add skip connection.
        return x + x_projected

### Build, train, and evaluate the gMLP model
 

In [17]:
gmlp_blocks = keras.Sequential(
    [gMLPLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.003
gmlp_classifier = build_classifier(gmlp_blocks)
history = run_experiment(gmlp_classifier)

Epoch 1/50
352/352 [==============================] - 46s 119ms/step - loss: 3.9187 - acc: 0.0963 - top5-acc: 0.2968 - val_loss: 3.5686 - val_acc: 0.1492 - val_top5-acc: 0.4146 - lr: 0.0030
Epoch 2/50
352/352 [==============================] - 42s 120ms/step - loss: 3.4666 - acc: 0.1659 - top5-acc: 0.4321 - val_loss: 3.3076 - val_acc: 0.1924 - val_top5-acc: 0.4740 - lr: 0.0030
Epoch 3/50
352/352 [==============================] - 41s 117ms/step - loss: 3.2458 - acc: 0.2078 - top5-acc: 0.4901 - val_loss: 3.1329 - val_acc: 0.2340 - val_top5-acc: 0.5266 - lr: 0.0030
Epoch 4/50
352/352 [==============================] - 41s 117ms/step - loss: 3.0660 - acc: 0.2410 - top5-acc: 0.5376 - val_loss: 2.9265 - val_acc: 0.2714 - val_top5-acc: 0.5716 - lr: 0.0030
Epoch 5/50
352/352 [==============================] - 41s 116ms/step - loss: 2.9380 - acc: 0.2639 - top5-acc: 0.5687 - val_loss: 2.8291 - val_acc: 0.2886 - val_top5-acc: 0.6008 - lr: 0.0030
Epoch 6/50
352/352 [==============================

## Conclusion
We can concluse that to get the better results for the three models (the MLP-Mixer, the FNet ,the gML ) we  can be achieved by increasing the embedding dimensions, increasing the number of (gMlp , FNet , mixer) blocks, and training the model for longer. You may also try to increase the size of the input images and use different patch sizes.